This project aims to create a neural network capable of estimating DoA from telecommunications signals.

To do so, a dataset consisting of In-Phase and Quadrature (I/Q)

In [1]:
import h5py
import numpy as np
import pandas as pd

In [2]:
import os
dataset_path = os.path.join(os.getcwd(),'DA-MUSIC','data','broad-band-synthetic-dataset.h5')

In [3]:
dataset_file = h5py.File(dataset_path, 'r')

In [4]:
dataset_file.keys()

<KeysViewHDF5 ['X', 'Y']>

In [5]:
dataset_file['X']

<HDF5 dataset "X": shape (10, 1, 200), type "<c16">

In [7]:
dataset_file['Y']

<HDF5 dataset "Y": shape (10, 1), type "<f8">

In [ ]:
dataset_file.close()